<a href="https://colab.research.google.com/github/agungsantoso/private-ai/blob/master/Section%202%20-%20Federated%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [0]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
!pip install syft
import syft as sy

     |████████████████████████████████| 225kB 5.0MB/s 
     |████████████████████████████████| 389kB 43.3MB/s 
     |████████████████████████████████| 204kB 47.0MB/s 
     |████████████████████████████████| 1.4MB 43.3MB/s 
     |████████████████████████████████| 256kB 45.0MB/s 
     |████████████████████████████████| 450kB 43.0MB/s 
     |████████████████████████████████| 81kB 25.2MB/s 
     |████████████████████████████████| 276kB 46.2MB/s 
     |████████████████████████████████| 51kB 19.5MB/s 
     |████████████████████████████████| 122kB 15.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

W0717 08:48:07.784892 140682828916608 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0717 08:48:07.803103 140682828916608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [12]:
bob._objects

{47686733235: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #objects:1>

In [14]:
x.id_at_location

47686733235

In [15]:
x.id

79273979718

In [16]:
x.owner

<VirtualWorker id:me #objects:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:79273979718 -> bob:47686733235]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr = x.send(bob, alice)

In [24]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [26]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [28]:
x

(Wrapper)>[PointerTensor | me:40425583006 -> bob:63519041415]

In [29]:
y

(Wrapper)>[PointerTensor | me:59517228175 -> bob:18324039139]

In [0]:
z = x + y

In [31]:
z

(Wrapper)>[PointerTensor | me:94717697710 -> bob:93843476700]

In [32]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [33]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:48498986313 -> bob:56750502689]

In [34]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [37]:
z.backward()

(Wrapper)>[PointerTensor | me:59673631940 -> bob:17550709192]

In [0]:
x = x.get()

In [39]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [40]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [43]:
for i in range(10):

    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects()

In [45]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [47]:
bob._objects

{28821090329: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [49]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [51]:
bob._objects

{27890796475: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [53]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [55]:
x

(Wrapper)>[PointerTensor | me:43842354525 -> bob:79611036282]

In [56]:
bob._objects

{79611036282: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [58]:
bob._objects

{79611036282: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [60]:
bob._objects

{79611036282: tensor([1, 2, 3, 4, 5])}

In [61]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [64]:
bob._objects

{37052575502: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [66]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [68]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [73]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(4.8018)
tensor(0.4236)
tensor(0.0776)
tensor(0.0398)
tensor(0.0283)
tensor(0.0210)
tensor(0.0158)
tensor(0.0119)
tensor(0.0090)
tensor(0.0068)
tensor(0.0051)
tensor(0.0039)
tensor(0.0029)
tensor(0.0022)
tensor(0.0017)
tensor(0.0013)
tensor(0.0010)
tensor(0.0008)
tensor(0.0006)
tensor(0.0004)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [78]:
train()

tensor(9.6533, requires_grad=True)
tensor(0.4615, requires_grad=True)
tensor(0.4610, requires_grad=True)
tensor(0.3949, requires_grad=True)
tensor(0.1891, requires_grad=True)
tensor(0.2346, requires_grad=True)
tensor(0.1067, requires_grad=True)
tensor(0.1362, requires_grad=True)
tensor(0.0616, requires_grad=True)
tensor(0.0789, requires_grad=True)
tensor(0.0356, requires_grad=True)
tensor(0.0457, requires_grad=True)
tensor(0.0206, requires_grad=True)
tensor(0.0265, requires_grad=True)
tensor(0.0119, requires_grad=True)
tensor(0.0154, requires_grad=True)
tensor(0.0069, requires_grad=True)
tensor(0.0089, requires_grad=True)
tensor(0.0040, requires_grad=True)
tensor(0.0052, requires_grad=True)
tensor(0.0023, requires_grad=True)
tensor(0.0030, requires_grad=True)
tensor(0.0013, requires_grad=True)
tensor(0.0017, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0010, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0003, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [79]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [82]:
bob._objects

{38149554374: tensor([1, 2, 3, 4, 5])}

In [83]:
alice._objects

{49481512756: (Wrapper)>[PointerTensor | alice:49481512756 -> bob:38149554374]}

In [0]:
y = x + x

In [85]:
y

(Wrapper)>[PointerTensor | me:19694459472 -> alice:15044166621]

In [86]:
bob._objects

{11175543829: tensor([ 2,  4,  6,  8, 10]),
 38149554374: tensor([1, 2, 3, 4, 5])}

In [87]:
alice._objects

{15044166621: (Wrapper)>[PointerTensor | alice:15044166621 -> bob:11175543829],
 49481512756: (Wrapper)>[PointerTensor | alice:49481512756 -> bob:38149554374]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [90]:
bob._objects

{28829788838: tensor([1, 2, 3, 4, 5])}

In [91]:
alice._objects

{2751762222: (Wrapper)>[PointerTensor | alice:2751762222 -> bob:28829788838]}

In [92]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:2751762222 -> bob:28829788838]

In [93]:
bob._objects

{28829788838: tensor([1, 2, 3, 4, 5])}

In [94]:
alice._objects

{}

In [95]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [96]:
bob._objects

{}

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [98]:
bob._objects

{59551609408: tensor([1, 2, 3, 4, 5])}

In [99]:
alice._objects

{85416971553: (Wrapper)>[PointerTensor | alice:85416971553 -> bob:59551609408]}

In [0]:
del x

In [101]:
bob._objects

{}

In [102]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [119]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [121]:
bob._objects

{5811013067: tensor([1, 2, 3, 4, 5])}

In [122]:
alice._objects

{}

In [123]:
x.move(alice)

(Wrapper)>[PointerTensor | me:58008875625 -> alice:58008875625]

In [124]:
bob._objects

{}

In [125]:
alice._objects

{58008875625: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [127]:
bob._objects

{81090552116: tensor([1, 2, 3, 4, 5])}

In [128]:
alice._objects

{58008875625: tensor([1, 2, 3, 4, 5]),
 70864437181: (Wrapper)>[PointerTensor | alice:70864437181 -> bob:81090552116]}

In [129]:
x.remote_get()

(Wrapper)>[PointerTensor | me:99079322792 -> alice:70864437181]

In [130]:
bob._objects

{}

In [131]:
alice._objects

{58008875625: tensor([1, 2, 3, 4, 5]), 70864437181: tensor([1, 2, 3, 4, 5])}

In [132]:
x.move(bob)

(Wrapper)>[PointerTensor | me:99079322792 -> bob:99079322792]

In [133]:
x

(Wrapper)>[PointerTensor | me:99079322792 -> bob:99079322792]

In [134]:
bob._objects

{99079322792: tensor([1, 2, 3, 4, 5])}

In [135]:
alice._objects

{58008875625: tensor([1, 2, 3, 4, 5])}